## Load Libraries

In [1]:
%matplotlib inline

import psycopg2
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pylab import figure, show
import holidays
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from scipy.stats import randint as sp_randint
from sklearn.grid_search import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")

## Connect to Postgres via psycopg and read in the 3 datasets into pandas dataframes

In [2]:
conn = psycopg2.connect(database="ontime", user="postgres",  host="localhost", port="5432")
delay = pd.read_sql_query("SELECT year, month, dayofmonth, dayofweek, flightdate, carrier, deptime, tailnum, flightnum, origin, dest, distancegroup, depdel15 FROM ontime_data LIMIT 500000;", conn)
delay.head(10)

,year,month,dayofmonth,dayofweek,flightdate,carrier,deptime,tailnum,flightnum,origin,dest,distancegroup,depdel15
0,2015,1,1,4,2015-01-01,AA,0855,N787AA,1,JFK,LAX,10,0.00
1,2015,1,2,5,2015-01-02,AA,0850,N795AA,1,JFK,LAX,10,0.00
2,2015,1,3,6,2015-01-03,AA,0853,N788AA,1,JFK,LAX,10,0.00
3,2015,1,4,7,2015-01-04,AA,0853,N791AA,1,JFK,LAX,10,0.00
4,2015,1,5,1,2015-01-05,AA,0853,N783AA,1,JFK,LAX,10,0.00
5,2015,1,6,2,2015-01-06,AA,0856,N799AA,1,JFK,LAX,10,0.00
6,2015,1,7,3,2015-01-07,AA,0859,N784AA,1,JFK,LAX,10,0.00
7,2015,1,8,4,2015-01-08,AA,0856,N787AA,1,JFK,LAX,10,0.00
8,2015,1,9,5,2015-01-09,AA,0901,N795AA,1,JFK,LAX,10,0.00
9,2015,1,10,6,2015-01-10,AA,0903,N790AA,1,JFK,LAX,10,0.00


## Load dataset with plane model information and dataset with airport (latitude, longitude) info

In [3]:
#Load the dataset with plane model information
planes = pd.read_sql_query("SELECT tailnum, manufacturer, model, engine_type, year FROM airplane_data", conn)

planes.rename(columns={'year' : 'planeyear'}, inplace=True)

print ("Length of dataframe - ", len(planes))
planes.head()

('Length of dataframe - ', 4480)


,tailnum,manufacturer,model,engine_type,planeyear
0,N10156,EMBRAER,EMB-145XR,Turbo-Fan,2004
1,N102UW,AIRBUS INDUSTRIE,A320-214,Turbo-Fan,1998
2,N10323,BOEING,737-3TO,Turbo-Jet,1986
3,N103US,AIRBUS INDUSTRIE,A320-214,Turbo-Fan,1999
4,N104UA,BOEING,747-422,Turbo-Fan,1998


In [4]:
#Load the dataset with plane model information
airport = pd.read_sql_query("SELECT iata, city, state, lat, long FROM airport_data", conn)

print ("Length of dataframe - ", len(airport))
airport.head()

('Length of dataframe - ', 3376)


,iata,city,state,lat,long
0,00M,Bay Springs,MS,31.953765,-89.234505
1,00R,Livingston,TX,30.685861,-95.017928
2,00V,Colorado Springs,CO,38.945749,-104.569893
3,01G,Perry,NY,42.741347,-78.052081
4,01J,Hilliard,FL,30.688012,-81.905944


## Add holiday feature

In [5]:
#Add a holiday variable to the delay dataset based on the variable "FlightDate"
us_holidays = holidays.UnitedStates()
holiday = delay.flightdate.apply(lambda x: x in us_holidays) # True/False
delay['holiday'] = LabelEncoder().fit_transform(holiday)
us_holidays

{datetime.date(2015, 1, 1): "New Year's Day",
 datetime.date(2015, 1, 19): 'Martin Luther King, Jr. Day',
 datetime.date(2015, 2, 16): "Washington's Birthday",
 datetime.date(2015, 5, 25): 'Memorial Day',
 datetime.date(2015, 7, 3): 'Independence Day (Observed)',
 datetime.date(2015, 7, 4): 'Independence Day',
 datetime.date(2015, 9, 7): 'Labor Day',
 datetime.date(2015, 10, 12): 'Columbus Day',
 datetime.date(2015, 11, 11): 'Veterans Day',
 datetime.date(2015, 11, 26): 'Thanksgiving',
 datetime.date(2015, 12, 25): 'Christmas Day'}

## Merge all three datasets into one

In [6]:
#Merge the the delay dataset with the planes dataset by "TailNum"
delay_planes = delay.merge(planes, left_on='tailnum', right_on='tailnum', how='inner')

#Merge the airport information to get the Lat-Long information on the origin airports
delay_planes.rename(columns={'origin' : 'iata'}, inplace=True)
delay_planes_origin = pd.merge(delay_planes, airport, on='iata', how = 'inner')
delay_planes_origin.rename(columns={'iata' : 'origin', 'airport' : 'origin_airport', 'city' : 'origin_city', 'state' : 'origin_state', 'lat' : 'origin_lat', 'long' : 'origin_long'}, inplace='True')

#Merge the airport information to get the Lat-Long information on the destination airports
delay_planes_origin.rename(columns={'dest' : 'iata'}, inplace=True)
final = pd.merge(delay_planes_origin, airport, on='iata', how = 'inner')
final.rename(columns={'iata' : 'dest', 'airport' : 'dest_airport', 'city' : 'dest_city', 'state' : 'dest_state', 'lat' : 'dest_lat', 'long' : 'dest_long'}, inplace='True')

In [7]:
final = final.rename(columns = {'model':'planemodel', 'engine_type':'enginetype'})
final = final[['year', 'month', 'dayofmonth', 'dayofweek', 'holiday', 'carrier', 'deptime', 'flightnum', 
               'origin', 'dest', 'distancegroup', 'planeyear', 'manufacturer', 'planemodel', 'enginetype', 
               'origin_lat', 'origin_long', 'dest_lat', 'dest_long', 'depdel15']]
final.head(10)

,year,month,dayofmonth,dayofweek,holiday,carrier,deptime,flightnum,origin,dest,distancegroup,planeyear,manufacturer,planemodel,enginetype,origin_lat,origin_long,dest_lat,dest_long,depdel15
0,2015,1,1,4,1,AA,1754,5,DFW,HNL,11,1978,PIPER,PA-32RT-300,Reciprocating,32.895951,-97.0372,21.318691,-157.922407,1.00
1,2015,1,19,1,1,AA,1051,123,DFW,HNL,11,1978,PIPER,PA-32RT-300,Reciprocating,32.895951,-97.0372,21.318691,-157.922407,0.00
2,2015,1,3,6,0,AA,1329,5,DFW,HNL,11,1992,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,1.00
3,2015,1,30,5,0,AA,1315,5,DFW,HNL,11,1992,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,0.00
4,2015,1,20,2,0,AA,1047,123,DFW,HNL,11,1992,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,0.00
5,2015,1,21,3,0,AA,1056,123,DFW,HNL,11,1992,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,0.00
6,2015,1,31,6,0,AA,1544,275,DFW,HNL,11,1992,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,0.00
7,2015,1,6,2,0,AA,1348,5,DFW,HNL,11,1988,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,1.00
8,2015,1,26,1,0,AA,1323,5,DFW,HNL,11,1988,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,0.00
9,2015,1,28,3,0,AA,1308,5,DFW,HNL,11,1988,BOEING,767-323,Turbo-Fan,32.895951,-97.0372,21.318691,-157.922407,0.00


## Visualize top 100 most popular flight route on the map

In [8]:
from IPython.display import HTML
import folium

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [9]:
#Aggregate to find most popular route
popular_route=final.groupby(['origin','dest','origin_lat','origin_long','dest_lat','dest_long'])['flightnum'].agg([len]).reset_index()

popular_route.reset_index(inplace=True)

popular_route=popular_route.sort_values(by='len', ascending=False)

In [10]:
#Plot the 100 most popular routes on the map
m = folium.Map(location=[41.9, -97.3], zoom_start=4)

flag=0

for index, row in popular_route.iterrows():
    orig=np.array([row['origin_lat'],row['origin_long']], dtype=float)
    dest=np.array([row['dest_lat'],row['dest_long']], dtype=float)
    flag=flag+1
    #  Create the map and add the line
    lines = folium.features.PolyLine([orig,dest], color='#883399', weight=2)
    m.add_children(lines)
    if flag==100:
        break

embed_map(m)

The above might not render properly on GitHub, so I have a screenshot of it in the GitHub repo as well.

# Prediction model

In [11]:
#Drop rows with NaN values
final = final.dropna()
print (final.shape)

(319603, 20)


## Shuffle the rows

In [12]:
# Set the randomizer seed so results are the same each time
np.random.seed(0)

#Shuffle the order of the rows
final = final.reindex(np.random.permutation(final.index))

#Check that the data frame is properly shuffled
final.head(10)

,year,month,dayofmonth,dayofweek,holiday,carrier,deptime,flightnum,origin,dest,distancegroup,planeyear,manufacturer,planemodel,enginetype,origin_lat,origin_long,dest_lat,dest_long,depdel15
184783,2015,1,22,4,0,US,1816,814,PHL,RDU,2,2007,EMBRAER,ERJ 190-100 IGW,Turbo-Fan,39.871953,-75.241141,35.877639,-78.787472,0.00
166903,2015,1,22,4,0,B6,1318,1170,MCO,RIC,3,2005,EMBRAER,ERJ 190-100 IGW,Turbo-Fan,28.428889,-81.316028,37.505167,-77.319667,0.00
219266,2015,1,25,7,0,OO,1504,4713,SLC,SMF,3,2006,BOMBARDIER INC,CL600-2D24,Turbo-Fan,40.788388,-111.977773,38.695422,-121.590767,0.00
230217,2015,1,18,7,0,DL,2008,1989,DTW,CLT,3,1998,AIRBUS INDUSTRIE,A320-212,Turbo-Fan,42.212059,-83.348836,35.214011,-80.943126,0.00
28286,2015,1,21,3,0,DL,1234,198,SEA,LAX,4,1997,BOEING,757-2Q8,Turbo-Fan,47.448982,-122.309313,33.942536,-118.408074,0.00
272362,2015,1,20,2,0,WN,1521,2291,AUS,DAL,1,2003,BOEING,737-7H4,Turbo-Fan,30.194533,-97.669872,32.847114,-96.851772,1.00
198327,2015,1,17,6,0,OO,1140,2646,ORD,ICT,3,2003,BOMBARDIER INC,CL-600-2B19,Turbo-Fan,41.979595,-87.904464,37.649959,-97.433046,0.00
219310,2015,1,30,5,0,DL,1940,1615,MSP,SMF,7,1992,AIRBUS INDUSTRIE,A320-211,Turbo-Jet,44.880547,-93.216922,38.695422,-121.590767,0.00
261466,2015,1,5,1,0,DL,0951,1082,SLC,OAK,3,1992,AIRBUS INDUSTRIE,A320-211,Turbo-Jet,40.788388,-111.977773,37.721291,-122.220717,0.00
159836,2015,1,29,4,0,EV,0520,4685,BRO,IAH,2,2000,EMBRAER,EMB-145LR,Turbo-Fan,25.906833,-97.425861,29.980472,-95.339722,0.00


## Recode categorical variables to numerical variables

In [13]:
#Recode string variables into numerical categorical variables
enc = LabelEncoder()
final['carrier'] = enc.fit_transform(final.carrier)
final['origin'] = enc.fit_transform(final.origin)
final['dest'] = enc.fit_transform(final.dest)
final['planeyear'] = enc.fit_transform(final.planeyear)
final['manufacturer'] = enc.fit_transform(final.manufacturer)
final['planemodel'] = enc.fit_transform(final.planemodel)
final['enginetype'] = enc.fit_transform(final.enginetype)

In [14]:
#Convert response variable to integer, it was loaded into Postgres as a varchar
depdel15_int=[int(x.split(".")[0]) for x in list(final.depdel15)]
final['depdel15_int'] = depdel15_int

## Split dataset into train, dev, and test set

In [15]:
#Separate the dataset into input and output columns
X = final[['year', 'month', 'dayofmonth', 'dayofweek', 'holiday', 'carrier', 'deptime', 'flightnum', 
           'origin', 'dest', 'distancegroup', 'planeyear', 'manufacturer', 'planemodel', 'enginetype']]
Y = final.depdel15_int

test_data, test_labels = X[250000:], Y[250000:]
dev_data, dev_labels = X[250000:251000], Y[250000:251000]
train_data, train_labels = X[:250000], Y[:250000]

In [16]:
#Check to see if the data is in the right shape
print ('Train data shape: ', train_data.shape)
print ('Test data shape: ', test_data.shape)

#Check percentage of delays in dev and test data, accuracy must be greater than these numbers
print ('Accuracy to beat(dev): ', 1-np.mean(dev_labels))
print ('Accuracy to beat(test): ', 1-np.mean(test_labels))

('Train data shape: ', (250000, 15))
('Test data shape: ', (69603, 15))
('Accuracy to beat(dev): ', 0.808)
('Accuracy to beat(test): ', 0.8079393129606482)


## Random forest with random gridsearch cross-validation

In [86]:
# This seed is required for each gridsearch to be identical
np.random.seed(0)

# Define the ML pipe with cross validation and parameters to optimize
pipe = Pipeline([("RF", RandomForestClassifier(n_jobs=1))])

params = {"RF__n_estimators": sp_randint(100, 200),
          "RF__max_depth": sp_randint(15, 50),
          "RF__min_samples_split": sp_randint(25, 45),
          "RF__min_samples_leaf": sp_randint(1, 30),
          "RF__max_leaf_nodes": sp_randint(5000, 7000),
          "RF__max_features": ["auto", .25, .5, .3]
         }

#Run a grid-search that chooses random paramaters based on the given range
gridsearch = RandomizedSearchCV(pipe, params, fit_params=None, n_iter=10, scoring="accuracy", cv=2, n_jobs=1)
gridsearch.fit(train_data, train_labels)

RandomizedSearchCV(cv=2, error_score='raise',
          estimator=Pipeline(steps=[('RF', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'RF__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000002868DDAC8>, 'RF__min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000002868DDE80>, 'RF__max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000... 'RF__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000028F43A0B8>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          sco

In [87]:
#Check the accuracy and f1-score on the dev_data
pred = gridsearch.predict(dev_data)
print ("Accuracy:", metrics.accuracy_score(dev_labels, pred))
print ("F1 score:", metrics.f1_score(dev_labels, pred))

Accuracy: 0.8366
F1 score: 0.214423076923


In [58]:
rf = RandomForestClassifier(n_estimators=500, max_depth=20)
rf.fit(train_data, train_labels)
print ("Accuracy", rf.score(dev_data, dev_labels))
pred = rf.predict(dev_data)
print "F1 score", metrics.f1_score(dev_labels, pred)

('Accuracy', 0.83499999999999996)
('F1 score', 0.36781609195402298)


## Assess predictive power of each feature

In [59]:
feature_importance = pd.DataFrame(rf.feature_importances_, columns=["Feature Importance"])
feature_importance.index = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Holiday', 'Carrier', 'DepTime', 'FlightNum', 
                            'Origin', 'Dest', 'DistanceGroup', 'PlaneYear', 'Manufacturer', 'PlaneModel', 'EngineType']
feature_importance.sort_values(by='Feature Importance', ascending=False)

,Feature Importance
DepTime,0.251040
DayofMonth,0.135266
FlightNum,0.132844
Origin,0.098586
Dest,0.093637
PlaneYear,0.074649
DistanceGroup,0.051812
DayOfWeek,0.047366
PlaneModel,0.045873
Carrier,0.026558


## Prediction example

In [60]:
predict = np.asarray([2015, 3, 4, 3, 0, 5, 1200, 111, 
                      25, 30, 11, 10, 1, 7, 5])

print ("Probability of delay:" , rf.predict_proba(predict.reshape(1,15))[0][1])

('Probability of delay:', 0.37363077731092431)


## Future goals
1. Deploy the prediction model onto an API where users can enter information on their upcoming flight and get a probability that the flight will be delayed. (See example above)

2. Identify a robust weather data source and incorporate that into our existing model.

3. Improve our current prediction model through future machine learning coursework in the MIDS program.